In [1]:
from models.CDAE import CDAE
from datasets.CiteYouLikeA import CiteYouLikeA

model = CDAE(CiteYouLikeA, sparse=False)

Importing Data ...
Done.


In [2]:
from models import utils
import tensorflow as tf

user_u = tf.placeholder(tf.int32, shape=[])

def _create_variables(n_hidden=50):
    global user_u
    weights = dict()
    weights['W'] = tf.Variable(utils.xavier_init(model.n_items, n_hidden))
    weights['W_prime'] = tf.Variable(utils.xavier_init(model.n_items, n_hidden))
    weights['b'] = tf.Variable(tf.zeros([1, n_hidden], dtype=tf.float32))
    weights['b_prime'] = tf.Variable(tf.zeros([1, model.n_items], dtype=tf.float32))
    weights['V'] = tf.Variable(tf.zeros([model.n_users, n_hidden]))
    weights['V_index'] = tf.nn.embedding_lookup(weights['V'], user_u)
    return weights

In [3]:
dropout_prob = tf.constant(0.2, dtype=tf.float32)
weight = _create_variables()




In [10]:
aug_set_indices = tf.placeholder(tf.int32, [None])
#n_hidden = 3

indices = [[i, j] for i in sess.run(aug_set_indices, feed_dict={aug_set_indices: [0, 2, 4]}) for j in xrange(n_hidden)]

#values = tf.reshape(grad, [-1])

#shape = [model.n_items, ]
#indices = [[]]
#tf.SparseTensor()


In [6]:
sess = tf.Session()
a = tf.constant([[1,2,34], [2,3,4], [9,6,2]])
b = sess.run(tf.reshape(a, [-1]))

In [ ]:
# METHOD 1 - NOT MEM
transfer_fn = tf.sigmoid                                                   
output_fn = tf.identity                                                        
n_hidden = 50                                                                    

# AdaGrad Parameters                                                                                                                   
ada_beta = 1.0                                                                   
ada_learning_rate = 0.01                                       
optimizer = tf.train.AdagradOptimizer(ada_learning_rate, ada_beta)                                                                    
#optimizer = tf.train.GradientDescentOptimizer(ada_learning_rate)                                                                       

# Regularization Parameter                                                                                                             
reg_lambda = 1.0                                                             



y = tf.placeholder(tf.float32, [None, model.n_items]) 
z_index = tf.placeholder(tf.int32, [])
selected_indices = tf.placeholder(tf.int32, [None])
grad_indices = tf.placeholder(tf.int64, [None, None])

z = tf.pack([transfer_fn(tf.add_n( [tf.matmul(y, weight['W']),                                                         
                                          tf.reshape(weight['V'][i], [1, n_hidden]),                                              
                                          weight['b']]                                                 
                                        )) for i in xrange(model.n_users) ])                                                     
y_hat = output_fn(tf.add(tf.matmul(tf.gather(z, z_index), tf.transpose(weight['W_prime'])),                                    
                                        weight['b_prime'])                                         
                                )

loss = tf.nn.l2_loss(tf.sub(y, y_hat))

dLoss_d = optimizer.compute_gradients(loss, var_list=[weight['W_prime'], weight['b_prime'], weight['W'], weight['V'], weight['b']])                                    

W_prime_loss_grads = tf.gather(dLoss_d[0][0], selected_indices)
b_prime_loss_grads = tf.gather(tf.transpose(dLoss_d[1][0]), selected_indices)
W_loss_grads = tf.gather(dLoss_d[2][0], selected_indices)
V_loss_grads = tf.gather(dLoss_d[3][0], selected_indices)
b_loss_grads = tf.gather(dLoss_d[4][0], selected_indices)                         

W_prime = tf.gather(weight['W_prime'], selected_indices)
b_prime = tf.gather(weight['b_prime'], selected_indices)
W = tf.gather(weight['W'], selected_indices)
V = tf.gather(weight['V'], selected_indices)
b = tf.gather(weight['b'], selected_indices)

#self.dLoss_d = lambda u, var: optimizer.compute_gradients(self.loss(u), var_list=[var])[0][0]
dCost_dW_prime = tf.mul(1.0 / model.n_users, W_prime_loss_grads) - tf.mul(reg_lambda, W_prime)
dCost_db_prime = tf.mul(1.0 / model.n_users, b_prime_loss_grads) - tf.mul(reg_lambda, b_prime)
dCost_dW = tf.mul(1.0 / model.n_users, W_loss_grads) - tf.mul(reg_lambda, W)
dCost_dV = tf.mul(1.0 / model.n_users, V_loss_grads) - tf.mul(reg_lambda, V)
dCost_db = tf.mul(1.0 / model.n_users, b_loss_grads) - tf.mul(reg_lambda, b)

#indices = [[i, j] for i in aug_set_indices for j in xrange(n_hidden)]


W_shape = [model.n_items, n_hidden]
V_shape = [model.n_users, n_hidden]
b_shape = [1, n_hidden]
b_prime_shape = [1, model.n_items]

cost_grad_W = tf.SparseTensor(indices=grad_indices, values=tf.reshape(dCost_dW, [-1]), shape=W_shape)
cost_grad_W_prime = tf.SparseTensor(indices=grad_indices, values=tf.reshape(dCost_dW_prime, [-1]), shape=W_shape)
cost_grad_b = tf.SparseTensor(indices=grad_indices, values=tf.reshape(dCost_db, [-1]), shape=b_shape)
cost_grad_b_prime = tf.SparseTensor(indices=grad_indices, values=tf.reshape(dCost_db_prime, [-1]), shape=b_prime_shape)
cost_grad_V = tf.SparseTensor(indices=grad_indices, values=tf.reshape(dCost_dV, [-1]), shape=V_shape)


                                     
cost_gradient_apply_W = optimizer.apply_gradients([[tf.sparse_tensor_to_dense(cost_grad_W), weight['W']]])
cost_gradient_apply_W_prime = optimizer.apply_gradients([[tf.sparse_tensor_to_dense(cost_grad_W_prime), weight['W_prime']]])
cost_gradient_apply_b = optimizer.apply_gradients([[tf.sparse_tensor_to_dense(cost_grad_b), weight['b']]])
cost_gradient_apply_b_prime = optimizer.apply_gradients([[tf.sparse_tensor_to_dense(cost_grad_b_prime), weight['b_prime']]])
cost_gradient_apply_V = optimizer.apply_gradients([[tf.sparse_tensor_to_dense(cost_grad_V), weight['V']]])
init_op = tf.initialize_all_variables()

In [4]:
# METHOD 1 - NOT MEM
transfer_fn = tf.sigmoid                                                   
output_fn = tf.identity                                                        
n_hidden = 50                                                                    

# AdaGrad Parameters                                                                                                                   
ada_beta = 1.0                                                                   
ada_learning_rate = 0.01                                       
optimizer = tf.train.AdagradOptimizer(ada_learning_rate, ada_beta)                                                                    
#optimizer = tf.train.GradientDescentOptimizer(ada_learning_rate)                                                                       

# Regularization Parameter                                                                                                             
reg_lambda = 1.0                                                             
prime_selected_indices = tf.placeholder(tf.int32, [None])
W_selected_indices = tf.placeholder(tf.int32, [None])
W_prime_indices = tf.placeholder(tf.int32, [None, None])
b_prime_indices = tf.placeholder(tf.int32, [None, None])
W_indices = tf.placeholder(tf.int32, [None, None])
user_indices = tf.placeholder(tf.int32, [None, None])

y = tf.placeholder(tf.float32, [None, model.n_items]) 


z = transfer_fn(tf.add_n([tf.matmul(y, weight['W']),                                                         
                                          tf.reshape(weight['V_index'], [1, n_hidden]),                                              
                                          weight['b']]                                                 
                                        ))                                         
y_hat = output_fn(tf.add(tf.matmul(z, tf.transpose(weight['W_prime'])),                                    
                                        weight['b_prime'])                                         
                                )

loss = tf.nn.l2_loss(tf.sub(y, y_hat))


# _train = optimizer.minimize(loss)
dLoss_d = optimizer.compute_gradients(loss, var_list=[weight['W_prime'], weight['b_prime'], weight['W'], weight['V'], weight['b']])                                    

W_prime_grads = tf.gather(dLoss_d[0][0], prime_selected_indices) + reg_lambda * tf.gather(weight['W_prime'], prime_selected_indices)
b_prime_grads = tf.gather(tf.transpose(dLoss_d[1][0]), prime_selected_indices) + reg_lambda * tf.gather(tf.transpose(weight['b_prime']), prime_selected_indices)
W_grads = tf.gather(dLoss_d[2][0], W_selected_indices) + reg_lambda * tf.gather(weight['W'], W_selected_indices)
V_grads = tf.gather(dLoss_d[3][0], user_u) + reg_lambda * tf.gather(weight['V'], user_u)
b_grads = dLoss_d[4][0] + reg_lambda * weight['b']


W_prime_values = tf.reshape(W_prime_grads, [-1])
b_prime_values = tf.reshape(b_prime_grads, [-1])
W_prime_grads = tf.sparse_to_dense(sparse_indices=W_prime_indices, output_shape=[model.n_items, n_hidden], sparse_values=W_prime_values)
b_prime_grads = tf.sparse_to_dense(sparse_indices=b_prime_indices, output_shape=[1, model.n_items], sparse_values=b_prime_values)

W_values = tf.reshape(W_grads, [-1])
W_grads = tf.sparse_to_dense(sparse_indices=W_indices, output_shape=[model.n_items, n_hidden], sparse_values=W_values)

V_values = tf.reshape(V_grads, [-1])
V_grads = tf.sparse_to_dense(sparse_indices=user_indices, output_shape=[model.n_users, n_hidden], sparse_values=V_values)
                       

_train = optimizer.apply_gradients([[W_prime_grads, weight['W_prime']],
                           [b_prime_grads, weight['b_prime']],
                           [W_grads, weight['W']],
                           [V_grads, weight['V']],
                           [b_grads, weight['b']]])

va

#self.dLoss_d = lambda u, var: optimizer.compute_gradients(self.loss(u), var_list=[var])[0][0

init_op = tf.initialize_all_variables()

In [5]:
# 1st iteration
sess = tf.Session()
sess.run(init_op)

input_data = model.input_data.get_data()

val_data = input_data[0].get_full_data()
train_data = input_data[0].get_train_data()


In [6]:
y_tilde = sess.run(tf.nn.dropout(tf.constant(train_data, dtype=tf.float32, shape=[1, model.n_items]), dropout_prob))

In [22]:
list(np.where(y_tilde > 0)[1])

[2317,
 3171,
 3404,
 3770,
 5114,
 5325,
 6103,
 7106,
 11226,
 11650,
 12802,
 13172,
 14851,
 15026,
 15833]

In [12]:
import numpy as np
augmented_O_set = list(model._create_augmented_O_set(np.array(val_data)))
indices = [[i, j] for i in augmented_O_set for j in xrange(n_hidden)]

In [18]:
print sorted(augmented_O_set)

[8, 11, 51, 70, 148, 186, 238, 253, 330, 337, 377, 420, 472, 484, 505, 519, 558, 706, 712, 773, 819, 839, 865, 879, 882, 1046, 1194, 1207, 1242, 1304, 1306, 1440, 1463, 1477, 1484, 1501, 1534, 1590, 1631, 1644, 1652, 1696, 1730, 1759, 1781, 1926, 1950, 2058, 2101, 2265, 2275, 2308, 2317, 2318, 2349, 2475, 2481, 2526, 2548, 2661, 2668, 2693, 2806, 2846, 2851, 2866, 2891, 2910, 2931, 3030, 3058, 3114, 3171, 3209, 3248, 3292, 3297, 3332, 3350, 3404, 3430, 3521, 3591, 3595, 3672, 3765, 3770, 3792, 3814, 3833, 3866, 3870, 3895, 3937, 3950, 3998, 4033, 4055, 4060, 4117, 4143, 4150, 4156, 4235, 4242, 4290, 4324, 4421, 4425, 4436, 4453, 4569, 4582, 4626, 4662, 4688, 4768, 4774, 4795, 4799, 4802, 4821, 4853, 4871, 4889, 4915, 4924, 4926, 4950, 5028, 5060, 5114, 5153, 5164, 5248, 5293, 5324, 5325, 5356, 5387, 5428, 5559, 5561, 5569, 5579, 5580, 5614, 5625, 5685, 5762, 5816, 5845, 5848, 5928, 5948, 5991, 5995, 6103, 6179, 6181, 6264, 6312, 6324, 6333, 6338, 6442, 6482, 6508, 6513, 6529, 6577, 662

In [7]:
prime_i = [0, 2]
W_prime_ind = [[i, j] for i in prime_i for j in xrange(n_hidden)]
b_prime_ind = [[0, 0], [0, 2]]

W_i = [1]
W_ind = [[i, j] for i in W_i for j in xrange(n_hidden)]

user_ind = [[0, i] for i in xrange(n_hidden)]

In [8]:
prev_weights = sess.run([weight['W'], weight['W_prime'], weight['b'], weight['b_prime'], weight['V']])

feed_dict={y: y_tilde, 
           user_u: 0, 
           prime_selected_indices: prime_i, 
           W_selected_indices: W_i, 
           W_prime_indices: W_prime_ind, 
           b_prime_indices: b_prime_ind, 
           W_indices: W_ind, 
           user_indices: user_ind}

sess.run(_train, feed_dict=feed_dict)
new_weights = sess.run([weight['W'], weight['W_prime'], weight['b'], weight['b_prime'], weight['V']])

In [9]:
print prev_weights[0]
print ''
print ''
print new_weights[0]

[[ 0.00033499 -0.01154972  0.00700667 ...,  0.0128306  -0.00399822
  -0.00866284]
 [ 0.01240425 -0.00628829 -0.01546988 ..., -0.01429989  0.0005925
   0.00754202]
 [ 0.01347434 -0.00643602  0.00626049 ..., -0.01472834  0.00265642
   0.0075183 ]
 ..., 
 [-0.00912714 -0.00035521  0.01465059 ..., -0.00507344 -0.00254842
   0.01498318]
 [-0.01274363 -0.01094532  0.01345679 ...,  0.01099324 -0.01446358
  -0.01169087]
 [ 0.00347038 -0.01392484 -0.00631091 ..., -0.01362248 -0.00043209
   0.00508399]]


[[ 0.00033499 -0.01154972  0.00700667 ...,  0.0128306  -0.00399822
  -0.00866284]
 [ 0.01228022 -0.00622541 -0.0153152  ..., -0.0141569   0.00058658
   0.0074666 ]
 [ 0.01347434 -0.00643602  0.00626049 ..., -0.01472834  0.00265642
   0.0075183 ]
 ..., 
 [-0.00912714 -0.00035521  0.01465059 ..., -0.00507344 -0.00254842
   0.01498318]
 [-0.01274363 -0.01094532  0.01345679 ...,  0.01099324 -0.01446358
  -0.01169087]
 [ 0.00347038 -0.01392484 -0.00631091 ..., -0.01362248 -0.00043209
   0.00508399]]

In [32]:
(reg_lambda *weight['b'])

<tf.Tensor 'mul_11:0' shape=(1, 50) dtype=float32>

In [28]:
weight['b'].get_shape()

TensorShape([Dimension(1), Dimension(50)])